In [7]:
from bs4 import BeautifulSoup
from datetime import timedelta
import dateparser
from requests import Session
from concurrent.futures import ThreadPoolExecutor
from requests_futures.sessions import FuturesSession


def fetch_raw_habr_pages_requests_futures(pages=10):
    ''' Получить сырые данные с хабра '''
    session = FuturesSession(
        executor=ThreadPoolExecutor(max_workers=20),
        session=Session()
    )
    pages_habr = []
    for page_number in range(1, pages+1):
        r = session.get(f"https://habr.com/ru/all/page{page_number}/",headers={
"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
})
        pages_habr.append(r)
    pages_habr = [r.result().text for r in pages_habr]
    return pages_habr


def fetch_raw_habr_pages(pages=10):
    ''' Получить сырые данные с хабра '''
    return fetch_raw_habr_pages_requests_futures(pages)


def convert_habr_date_to_datetime(date_habr):
    ''' Конвертер строковой даты с habr.com  в datetime '''
    return dateparser.parse(date_habr)


def get_titles_articles_with_raw_habr_pages(habr_pages):
    ''' Распарсить сырые страницы: выбрать заголовки статей с датами
        публикации '''
    titles_articles = []
    for page_text in habr_pages:
        soup = BeautifulSoup(page_text, "html.parser")
        
        articles = soup.find_all("div", class_="tm-article-snippet tm-article-snippet")
        for article in articles:
            date_of_publication_tag = article.find("time")
    
            if not date_of_publication_tag:
                continue
            # Конвертируем её в datetime
            date_of_publication = convert_habr_date_to_datetime(
                date_of_publication_tag.get_text()
            )
            
            title_article_tag = article.find("a", class_="tm-article-snippet__title-link")
            title_article_tag = title_article_tag.find("span")
            if not title_article_tag:
                continue

            title_article = title_article_tag.get_text()

            titles_articles.append(
                {'date': date_of_publication,
                 'title': title_article}
            )
    return titles_articles


def get_weeks(date_begin, date_end):
    # Формируем список недель
    data_cur = date_begin - timedelta(days=date_begin.weekday())
    delta = date_end - data_cur
    return [
        (data_cur + timedelta(days=i), data_cur + timedelta(days=i+1))
        for i in range(0, delta.days, 1)
    ]


def divide_titles_at_weeks(titles_articles,):
    ''' Разделить заголовки по неделям '''
    if not titles_articles:
        return []
    # Берём даты публикаций самой старой и самой свежей статьи
    date_begin = titles_articles[-1]['date'].date()
    date_end = titles_articles[0]['date'].date()

    # Формируем список недель
    weeks = get_weeks(date_begin, date_end)

    # Разбиваем статьи по неделям
    titles_articles_weeks = []
    for date_begin_week, date_end_week in weeks:
        titles_articles_weeks.append({
            'date': date_begin_week,
            'titles_articles': [
                title_article for title_article in titles_articles
                if (title_article['date'].date() >= date_begin_week and
                    title_article['date'].date() < date_end_week)
            ]
        })
    return titles_articles_weeks

In [8]:
import pymorphy2
import re
import collections


def flat(not_flat_list):
    """ [(1,2), (3,4)] -> [1, 2, 3, 4]"""
    return [item for sublist in not_flat_list for item in sublist]


def parse_nouns_in_titles_articles(titles_articles, morph=None):
    ''' Выбрать существительные.'''

    # Разбиваем заголовок на слова
    #Очищаем строку от "мусора"
    words = flat([
        re.sub('[^a-zA-Zа-яА-Я0-9 -]', '', title_article['title']).split()
        for title_article in titles_articles['titles_articles']
    ])

    if not morph:
        morph = pymorphy2.MorphAnalyzer()

    nouns = []
    for word in words:
        # Морфологический разбор
        word_parses = morph.parse(word)
        if word_parses:
            if ('NOUN' in word_parses[0].tag or
                    'LATN' in word_parses[0].tag):
                nouns.append(word_parses[0].normal_form)
    return nouns


def parse_nouns_in_titles_articles_at_weeks(titles_articles_weeks, top_size):
    '''Выбрать самые популярные существительные по неделям.'''
    top_nouns_weeks = []
    morph = pymorphy2.MorphAnalyzer()
    for titles_articles in titles_articles_weeks:
        nouns = parse_nouns_in_titles_articles(titles_articles, morph)
        top_nouns_weeks.append({
            'date': titles_articles['date'],
            'top_words': get_top_words(nouns, top_size),
        })
    return top_nouns_weeks


def get_top_words(words, top_size=10):
    ''' Сформировать ТОП слов '''
    return collections.Counter(words).most_common(top_size)

In [12]:
import sys
from texttable import Texttable
import argparse

def output_words_stat(nouns_weeks):
    # Инициализируем таблицу на печать
    table = Texttable()
    table.set_cols_align(['c', 'l'])
    table.set_cols_valign(['m', 'm'])
    table.header(['Начало недели', 'Популярные слова'])
    for nouns_week in nouns_weeks:
        top_words = nouns_week['top_words']
        table.add_row((
            nouns_week['date'],
            ', '.join(['%s (%d)' % (noun, count) for noun, count in top_words])
        ))
    # Прорисовываем таблицу
    print(table.draw())





pages_count = 10
top_size = 20

habr_pages = fetch_raw_habr_pages(pages_count)
print('получили {0} страниц с habr.com'.format(len(habr_pages)))

titles_articles = get_titles_articles_with_raw_habr_pages(habr_pages)
print('количество статей: {0}'.format(len(titles_articles)))

titles_articles_weeks = divide_titles_at_weeks(titles_articles)

top_nouns_weeks = parse_nouns_in_titles_articles_at_weeks(
    titles_articles_weeks, top_size
)

output_words_stat(top_nouns_weeks)



получили 10 страниц с habr.com
количество статей: 200
+---------------+--------------------------------------------------------------+
| Начало недели |                       Популярные слова                       |
+===============+==============================================================+
|  2023-01-09   |                                                              |
+---------------+--------------------------------------------------------------+
|  2023-01-10   | pytorch (1), цепочка (1), зависимость (1), дельта (1),       |
|               | компрессия (1), квантизация (1), объект (1), c (1)           |
+---------------+--------------------------------------------------------------+
|               | python (6), сеть (3), функция (2), собеседование (2), бот    |
|               | (2), контейнер (2), будущее (2), анализ (2), работа (2),     |
|  2023-01-11   | падение (1), продажа (1), пк (1), ноутбук (1), прогноз (1),  |
|               | аналитик (1), chatgpt (1), задание (1